In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor


In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample = pd.read_csv("data/sample_submission.csv")

train.head()


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [3]:
X=train.drop(columns=['accident_risk'],axis=1)
y=train['accident_risk']


In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [5]:
cat_features=X.select_dtypes(include='O').columns
num_features=X.select_dtypes(exclude='O').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ("OneHoteEncoder",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features)
    ]
)
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [6]:
!pip install lightgbm

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
models={
    "Gradient_Boosting":GradientBoostingRegressor()
}

for i in range(len(models)):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    #Train Performance
    model_train_mse = mean_squared_error(y_train, y_train_pred)
    model_train_mae = mean_absolute_error(y_train, y_train_pred)
    model_train_r2 = r2_score(y_train, y_train_pred)

    # Test Performance
    model_test_mse = mean_squared_error(y_test, y_test_pred)
    model_test_mae = mean_absolute_error(y_test, y_test_pred)
    model_test_r2 = r2_score(y_test, y_test_pred)

    print(list(models.keys())[i])  #Print all model name

print('Model Performance on Training Data')
print('----------------------------------')
print("-Mean Squared Error (MSE): {:.4f}".format(model_train_mse))
print("-Root Mean Squared Error (RMSE): {:.4f}".format(np.sqrt(model_train_mse)))
print("-Mean Absolute Error (MAE): {:.4f}".format(model_train_mae))
print("-R2 Score: {:.4f}".format(model_train_r2))

print('\nModel Performance on Validation Data')
print('----------------------------------')
print("-Mean Squared Error (MSE): {:.4f}".format(model_test_mse))
print("-Root Mean Squared Error (RMSE): {:.4f}".format(np.sqrt(model_test_mse)))
print("-Mean Absolute Error (MAE): {:.4f}".format(model_test_mae))
print("-R2 Score: {:.4f}".format(model_test_r2))

print('='*35)
print('\n')


Gradient_Boosting
Model Performance on Training Data
----------------------------------
-Mean Squared Error (MSE): 0.0032
-Root Mean Squared Error (RMSE): 0.0569
-Mean Absolute Error (MAE): 0.0443
-R2 Score: 0.8833

Model Performance on Validation Data
----------------------------------
-Mean Squared Error (MSE): 0.0032
-Root Mean Squared Error (RMSE): 0.0570
-Mean Absolute Error (MAE): 0.0444
-R2 Score: 0.8824




In [8]:
X = train.drop('accident_risk', axis=1)
y = train['accident_risk']


In [9]:
cat_features = X.select_dtypes(include='object').columns
num_features = X.select_dtypes(exclude='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)



In [10]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [11]:
X_train_p = preprocessor.fit_transform(X_train)
X_val_p = preprocessor.transform(X_val)


In [12]:
xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",
    random_state=42
)
xgb_model.fit(X_train_p, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.03, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [21]:
xgb_test_preds = xgb_model.predict(test_processed)
xgb_test_preds = np.clip(xgb_test_preds, 0, 1)


NameError: name 'test_processed' is not defined

In [22]:
submission = sample.copy()
submission["accident_risk"] = xgb_test_preds
submission.to_csv("submission_data.csv", index=False)
submission.head()


NameError: name 'xgb_test_preds' is not defined

In [20]:
import joblib
import os
y_pred = xgb_model.predict(X_val_p)
print("Validation MSE:", mean_squared_error(y_val, y_pred))
print("Validation MAE:", mean_absolute_error(y_val, y_pred))
print("Validation R2:", r2_score(y_val, y_pred))

# ---- OPTIONAL: create submission if test present ----
if test is not None and sample is not None:
    test_p = preprocessor.transform(test)
    test_preds = np.clip(xgb_model.predict(test_p), 0, 1)
    submission = sample.copy()
    submission["accident_risk"] = test_preds
    submission.to_csv("submission_data.csv", index=False)
    print("Wrote submission_data.csv")

# ---- SAVE ARTIFACTS ----
joblib.dump(xgb_model, os.path.join("c:/Users/Admin/Desktop/Data Science/My Learning/Kaggle/models", "model.joblib"))
joblib.dump(preprocessor, os.path.join("c:/Users/Admin/Desktop/Data Science/My Learning/Kaggle/models", "preprocessor.joblib"))
print("Saved model and preprocessor into 'models/'")


Validation MSE: 0.0031723698615496653
Validation MAE: 0.04374253191342678
Validation R2: 0.8851098239003555
Wrote submission_data.csv
Saved model and preprocessor into 'models/'
